**WARNING**: This notebook does not work anymore as it relies on loading the models as keras models. But the old models were saved as keras models, they cannot be loaded as keras 3 models, having to be loaded with tf.saved_model.load, which does not expose layers.

**DECISION**: We will not fix this notebook. If we want to conduct more experiments with the latent space exploration, we will need to save the models trained with tf>2.10 using the keras save format (on the multi-domain repository), and change this notebook to load as such accordingly.

# Exploring the Latent Space of the Models

Here we explore the latent space of the trained models. We will use the trained models to generate new samples and visualize the latent space of the models. We will also use the latent space to interpolate between two samples and visualize the interpolation.

But first, we start by finding the most distant (in latent space) samples in the dataset (train and test) and visualize them.


In [1]:
import gc
import logging
from time import time
import tensorflow as tf

import logging_utils

from ModelProxy import CollaGANModelProxy, Pix2PixModelProxy, StarGANModelProxy

logging_utils.configure()

model_loaders = {
    'Pix2Pix': lambda: Pix2PixModelProxy('models/pix2pix'),
    'StarGAN': lambda: StarGANModelProxy('models/stargan'),
    'CollaGAN': lambda: CollaGANModelProxy('models/collagan'),
}


2025-05-29 18:47:11.133587: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-29 18:47:11.156614: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-29 18:47:11.156655: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-29 18:47:11.171248: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-29 18:47:22.046211: W tensorflow/compiler/tf

In [2]:
from dataset_utils import DatasetLoader

dataset_loader = DatasetLoader("tiny-hero", "test", limit=None)


## Finds the Distance Between Two Samples in the Latent Space


In [3]:
from scipy.spatial import distance
import numpy as np

def calculate_codes(model, dataset_loader, batch_size=128):
    codes = []
    for samples in dataset_loader.dataset.batch(batch_size):
        z = model.encode(2, 3, samples)
        codes.append(z)

    codes = np.concatenate(codes, axis=0)
    codes = np.reshape(codes, (dataset_loader.dataset_size, -1))
    return codes

def find_most_distant_codes(codes):
    distances = distance.cdist(codes, codes, "sqeuclidean")
    maxarg = np.unravel_index(distances.argmax(), distances.shape)

    return maxarg, distances



# model = model_loaders["StarGAN"]()
model = model_loaders["Pix2Pix"]()
# do a cold start of the model to have tf know which are the model inputs and outputs,
# which is required for the StarGAN's surrogate model to properly work
model.generate(2, 3, tf.random.uniform(shape=[1, 4, 64, 64, 4], minval=-1., maxval=1.))
samples = []
for dataset_name in [
    # "tiny-hero",
    "rpg-maker-2000",
    "rpg-maker-xp",
    "rpg-maker-vxace"
]:
    print(f"Processing ds {dataset_name}...")
    dataset = DatasetLoader(dataset_name, "train", limit=None)
    codes = calculate_codes(model, dataset)
    maxarg, distances = find_most_distant_codes(codes)
    samples = samples + [dataset.load_paired_images(maxarg[0]), dataset.load_paired_images(maxarg[1])]
    print("len(samples)", len(samples))
    del codes


2025-05-29 18:47:42 INFO     Start >> Loading Pix2Pix model front-to-right
2025-05-29 18:47:45 INFO     Fingerprint not found. Saved model loading will continue.
2025-05-29 18:47:45 INFO     path_and_singleprint metric could not be logged. Saved model loading will continue.
2025-05-29 18:47:45 INFO     End   >> Loading Pix2Pix model front-to-right
2025-05-29 18:47:45.604864: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
2025-05-29 18:47:48.549169: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.67GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


Processing ds rpg-maker-2000...


AttributeError: 'MultiInputTFSMLayer' object has no attribute 'inputs'

In [ ]:
from matplotlib import pyplot as plt


# shows an image with the four different directions in the rows and the train and test images in the columns
rows = 4
cols = len(samples)
fig = plt.figure(figsize=(4*cols, 4*rows))
for i in range(rows):
    for j in range(cols):
        plt.subplot(rows, cols, i*cols + j + 1, facecolor='w')
        plt.imshow(samples[j][i] * 0.5 + 0.5, interpolation="nearest")
        plt.axis("off")
fig.patch.set_alpha(0.0)
plt.tight_layout()
plt.show()
plt.savefig("output/most-distant-samples-per-dataset.png", transparent=True)